In [1]:
%env OMP_NUM_THREADS=10

env: OMP_NUM_THREADS=10


In [2]:
%matplotlib inline
import os
import sys
import pandas as pd
import numpy as np
import turicreate as tc

/home/kdomokos/anaconda3/envs/py3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
from alpenglow.experiments import ExternalModelExperiment
from alpenglow.evaluation import DcgScore

In [4]:
i=40

In [5]:
batch = pd.read_csv("batches/batch_%d_train.dat" % i)

In [6]:
test_users = pd.read_csv("batches/batch_%d_test.dat" % i)

In [7]:
model = tc.ranking_factorization_recommender.create(
    tc.SFrame(batch),
    'user',
    'item',
    num_factors=10,
    verbose=True,
    solver='ials',
    max_iterations=50,
    ials_confidence_scaling_factor=20
)
results = model.recommend(users=test_users.user.values, k=100, exclude_known=True)

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 444492 observations with 36722 users and 43825 items.

Data prepared in: 1.02828s

Training ranking_factorization_recommender for recommendations.

...

recommendations finished on 1000/5206 queries. users per second: 2131.43

recommendations finished on 2000/5206 queries. users per second: 2125.76

recommendations finished on 3000/5206 queries. users per second: 2071.96

recommendations finished on 4000/5206 queries. users per second: 2046.95

recommendations finished on 5000/5206 queries. users per second: 1961.63

In [8]:
results.to_dataframe()[['user','item','rank']].to_csv('batches/batch_%d_predictions.dat' % i, sep=' ', header=False)

In [9]:
data = pd.read_csv('../tutorial_data/data.csv', header=None, names=['time', 'user', 'item'])

exp = ExternalModelExperiment(
    period_length=60 * 60 * 24 * 7,
    in_name_base="batches/batch",
    mode="read",
)

In [10]:
res = exp.run(data)

running experiment...


In [11]:
res['dcg'] = DcgScore(res)

In [12]:
res['dcg'].groupby(res['time']//(7*24*60*60)).mean()[2338.0]

0.014344115488999583